In [ ]:
!pip install node2vec
!pip install --upgrade gensim
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=e1b64ebc4de34e05bdb8176898bb1178291d37b2906763d0c98f42e606d9ed22
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec
     |████████████████████████████████| 24.1 MB 78.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 7.9 MB 35.3 MB/s 
     |████████████████████████████████| 3.5 MB 30.6 MB/s 


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
import numpy as np 
import random
random.seed(2022)
np.random.seed(2022)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_10000_9.csv to data_10000_9.csv
Saving edge_index_data_9.csv to edge_index_data_9.csv


In [ ]:
uploaded2 = files.upload()

Saving deepwalk_10d_x9.model to deepwalk_10d_x9.model
Saving Node2Vec_10d_x9.model to Node2Vec_10d_x9.model


In [ ]:
uploaded3 = files.upload()

Saving feats_ate_x9.xlsx to feats_ate_x9.xlsx


In [ ]:
import io
df=pd.read_csv(io.BytesIO(uploaded['data_10000_9.csv']),index_col=0)

from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size = 0.2,random_state=42)
df

,X0,X1,X2,X3,X4,X5,X6,X7,X8,T,y,tau,b,e
0,0.609399,0.617459,0.226921,0.004359,0.661646,0.587542,0.426148,0.705872,0.073834,1,1.006577,0.613429,1.409735,0.900000
1,0.835325,0.171336,0.723448,0.643941,0.351913,0.923063,0.628349,0.513777,0.865254,0,2.414745,0.503331,1.354386,0.434631
2,0.640500,0.000786,0.625402,0.270436,0.083907,0.505043,0.057420,0.315679,0.406920,0,2.194673,0.320643,0.345424,0.100000
3,0.108567,0.594597,0.530164,0.261543,0.733564,0.454656,0.623660,0.301259,0.325126,0,0.845496,0.351582,0.831559,0.201414
4,0.534229,0.438480,0.679938,0.074687,0.071026,0.760965,0.131953,0.041444,0.336757,1,2.220674,0.486354,0.846220,0.671265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.886510,0.195066,0.283373,0.465533,0.482518,0.099990,0.010150,0.900044,0.546843,1,1.272802,0.540788,1.317584,0.516938
9996,0.993493,0.414716,0.167049,0.898168,0.883686,0.555195,0.939795,0.446944,0.670944,1,2.604282,0.704104,2.523766,0.900000
9997,0.864848,0.326439,0.325487,0.126372,0.429563,0.997772,0.174929,0.967537,0.588439,1,1.822956,0.595644,1.177203,0.775139
9998,0.843089,0.690255,0.432493,0.709480,0.033376,0.687961,0.781540,0.317079,0.218253,1,2.139239,0.766672,1.702327,0.900000


In [ ]:
col=df.columns
y_train=torch.from_numpy(np.array((df_train['y']))).reshape(df_train.shape[0],1).to(torch.float32)
y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_train=torch.from_numpy(np.array(df_train[col[:10]])).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[col[:10]])).to(torch.float32)

train_reverse=df_train[col[:10]].copy()
train_reverse["T"]=1-train_reverse["T"]
train_reverse=torch.from_numpy(np.array(train_reverse)).to(torch.float32)
test_reverse=df_test[col[:10]].copy()
test_reverse["T"]=1-test_reverse["T"]
test_reverse=torch.from_numpy(np.array(test_reverse)).to(torch.float32)

In [ ]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(p, q):

    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl 

In [ ]:
edge_index=pd.read_csv(io.BytesIO(uploaded['edge_index_data_9.csv']))
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))

In [ ]:
from gensim.models import word2vec
model_dw=word2vec.Word2Vec.load("deepwalk_10d_x9.model")
model_n2v=word2vec.Word2Vec.load("Node2Vec_10d_x9.model")

lst_dw=[]
lst_n2v=[]
for i in range(10):
    lst_dw.append(model_dw.wv[i])
    lst_n2v.append(model_n2v.wv[i])
len(lst_n2v)

10

In [ ]:
ate_list=[]
ate=pd.read_excel(io.BytesIO(uploaded3["feats_ate_x9.xlsx"]),index_col=0)
for i in col[:10]:
    ate_list.append(float(ate[ate['Feature']==i]["ATE"].values))
 
ate_list

[0.5712448107497264,
 0.582576493290745,
 -0.0373163925354057,
 0.9667129037810936,
 0.4958998601192993,
 -0.005398536686650696,
 0.01784809067518973,
 0.02109120439147628,
 0.03971268021246396,
 0.4949991890269561]

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url

# create Mytrain Data
class Mytrain(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_train.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_train[i]
            t=torch.zeros(10,21) # 21 dimensions = 1+10+10 = node number, node embedding, weight*feature

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(10):
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 
              
            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytest Data
class Mytest(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_test.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_test[i]
            t=torch.zeros(10,21) # 21 dimensions = 1+10+10 = node number, node embedding, weight*feature

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(10):
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 


            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytrain_reverse Data
class Mytrain_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(train_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =train_reverse[i]
            t=torch.zeros(10,21) # 21 dimensions = 1+10+10 = node number, node embedding, weight*feature

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(10):
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 

            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytest_reverse Data
class Mytest_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(test_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =test_reverse[i]
            t=torch.zeros(10,21) # 21 dimensions = 1+10+10 = node number, node embedding, weight*feature

            causal_weighted= np.multiply(np.array(X),np.array(ate_list))
            weighted_feats.append(causal_weighted)
            
            for j in range(10):
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec
                                                     list(causal_weighted) # weight*feature (6d), causal-weighted feature        
                                                    ))) 

            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
Mydata_train=Mytrain(".\mydata\MYdata_train")
Mydata_test=Mytest(".\mydata\MYdata_test")
Mytrain_reverse=Mytrain_reverse(".\mydata\MYtrain_reverse")
Mytest_reverse=Mytest_reverse(".\mydata\MYtest_reverse")

Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!


# GCN

In [ ]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(21, 64)
        self.conv2 = GCNConv(64, 11)
        self.f1 = torch.nn.Linear(110,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,110)
        x = self.f1(x)
        x = self.f2(x)
        return x

In [ ]:
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    

te_train = {}
te_valid = {}
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GCN ATE % Error",i]=np.mean(train_error)
    te_train["GCN MSE",i]=np.mean(train_mse)
    te_train["GCN KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GCN ATE % Error",i]=np.mean(valid_error)
    te_valid["GCN MSE",i]=np.mean(valid_mse)
    te_valid["GCN KL Divergence",i]=np.mean(valid_kl)
    print(te_valid)

R2:-0.396249  MSE:1.973560  KL:0.243632  MAE:1.124628  RMSE:1.404834  CVRMSE:0.933018  train_loss:
1.9735599851608276
R2:0.086275  MSE:1.326699  KL:1.036618  MAE:0.924824  RMSE:1.151824  CVRMSE:0.811379  test_loss:
0.0026507665514945984
R2:0.090686  MSE:1.285291  KL:0.710376  MAE:0.904326  RMSE:1.133707  CVRMSE:0.752949  R2:0.148240  MSE:1.203940  KL:0.515767  MAE:0.875248  RMSE:1.097242  CVRMSE:0.728731  R2:0.186084  MSE:1.150448  KL:0.390904  MAE:0.850455  RMSE:1.072590  CVRMSE:0.712358  R2:0.195745  MSE:1.136792  KL:0.296151  MAE:0.847910  RMSE:1.066205  CVRMSE:0.708118  R2:0.202105  MSE:1.127803  KL:0.282383  MAE:0.845520  RMSE:1.061981  CVRMSE:0.705312  R2:0.211486  MSE:1.114544  KL:0.264386  MAE:0.840819  RMSE:1.055720  CVRMSE:0.701154  R2:0.214492  MSE:1.110295  KL:0.259970  MAE:0.839619  RMSE:1.053705  CVRMSE:0.699816  R2:0.219359  MSE:1.103414  KL:0.252274  MAE:0.834931  RMSE:1.050435  CVRMSE:0.697645  R2:0.220917  MSE:1.101212  KL:0.248490  MAE:0.835713  RMSE:1.049387  CVRMSE

In [ ]:
te_train,te_valid

({('GCN ATE % Error', 0): 0.05406489672353565,
  ('GCN ATE % Error', 1): 0.022293245014452845,
  ('GCN ATE % Error', 2): 0.050501173172270875,
  ('GCN ATE % Error', 3): 0.017282902942943767,
  ('GCN ATE % Error', 4): 0.03924306088933438,
  ('GCN KL Divergence', 0): 0.4874352631142574,
  ('GCN KL Divergence', 1): 0.46805876146859726,
  ('GCN KL Divergence', 2): 0.46366734365923973,
  ('GCN KL Divergence', 3): 0.45572964721404907,
  ('GCN KL Divergence', 4): 0.5443725442198593,
  ('GCN MSE', 0): 0.16350285474953166,
  ('GCN MSE', 1): 0.1557692647556557,
  ('GCN MSE', 2): 0.15998502721453622,
  ('GCN MSE', 3): 0.13641157289123593,
  ('GCN MSE', 4): 0.19564469853939145},
 {('GCN ATE % Error', 0): 0.1542615423126602,
  ('GCN ATE % Error', 1): 0.13336418777453019,
  ('GCN ATE % Error', 2): 0.04266630267155491,
  ('GCN ATE % Error', 3): 0.11276911294842995,
  ('GCN ATE % Error', 4): 0.03721466040690791,
  ('GCN KL Divergence', 0): 0.4962411869100533,
  ('GCN KL Divergence', 1): 0.454604176216

# GAT

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=21,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=11,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(110,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,110)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GAT ATE % Error",i]=np.mean(train_error)
    te_train["GAT MSE",i]=np.mean(train_mse)
    te_train["GAT KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GAT ATE % Error",i]=np.mean(valid_error)
    te_valid["GAT MSE",i]=np.mean(valid_mse)
    te_valid["GAT KL Divergence",i]=np.mean(valid_kl)


R2:-0.378403  MSE:1.948335  KL:0.126827  MAE:1.108526  RMSE:1.395828  CVRMSE:0.927036  train_loss:
1.9483350009918212
R2:0.122150  MSE:1.274610  KL:0.575823  MAE:0.899032  RMSE:1.128986  CVRMSE:0.795292  test_loss:
0.002545548677444458
R2:0.106638  MSE:1.262743  KL:0.284303  MAE:0.896958  RMSE:1.123718  CVRMSE:0.746315  R2:0.159376  MSE:1.188199  KL:0.271141  MAE:0.868514  RMSE:1.090046  CVRMSE:0.723952  R2:0.168852  MSE:1.174806  KL:0.228243  MAE:0.860993  RMSE:1.083884  CVRMSE:0.719860  R2:0.157728  MSE:1.190528  KL:0.228787  MAE:0.867737  RMSE:1.091113  CVRMSE:0.724661  R2:0.165339  MSE:1.179770  KL:0.229948  MAE:0.865269  RMSE:1.086172  CVRMSE:0.721379  R2:0.161994  MSE:1.184499  KL:0.230475  MAE:0.865407  RMSE:1.088347  CVRMSE:0.722823  R2:0.173180  MSE:1.168687  KL:0.236551  MAE:0.858319  RMSE:1.081058  CVRMSE:0.717983  R2:0.180630  MSE:1.158158  KL:0.229980  MAE:0.857196  RMSE:1.076178  CVRMSE:0.714741  R2:0.187989  MSE:1.147756  KL:0.230127  MAE:0.852508  RMSE:1.071334  CVRMSE:

In [ ]:
te_train,te_valid

({('GAT ATE % Error', 0): 0.2945428181360128,
  ('GAT ATE % Error', 1): 0.2590877223639798,
  ('GAT ATE % Error', 2): 0.259875619706569,
  ('GAT ATE % Error', 3): 0.26107370052801837,
  ('GAT ATE % Error', 4): 0.263214159753294,
  ('GAT KL Divergence', 0): 0.4401926644446653,
  ('GAT KL Divergence', 1): 0.4038325129467251,
  ('GAT KL Divergence', 2): 0.40616598445958757,
  ('GAT KL Divergence', 3): 0.4225133434364956,
  ('GAT KL Divergence', 4): 0.41759226783393766,
  ('GAT MSE', 0): 0.07746518520991819,
  ('GAT MSE', 1): 0.07089239413562756,
  ('GAT MSE', 2): 0.07156449299034072,
  ('GAT MSE', 3): 0.07082810931753797,
  ('GAT MSE', 4): 0.07094615422862927,
  ('GCN ATE % Error', 0): 0.05406489672353565,
  ('GCN ATE % Error', 1): 0.022293245014452845,
  ('GCN ATE % Error', 2): 0.050501173172270875,
  ('GCN ATE % Error', 3): 0.017282902942943767,
  ('GCN ATE % Error', 4): 0.03924306088933438,
  ('GCN KL Divergence', 0): 0.4874352631142574,
  ('GCN KL Divergence', 1): 0.46805876146859726,

In [ ]:
gcn_error=[te_train[i] for i in te_train if 'GCN ATE % Error' in i[0]][2]
gcn_mse=[te_train[i] for i in te_train if 'GCN MSE' in i[0]][2]
gcn_kl=[te_train[i] for i in te_train if 'GCN KL Divergence' in i[0]][2]
gat_error=[te_train[i] for i in te_train if 'GAT ATE % Error' in i[0]][1]
gat_mse=[te_train[i] for i in te_train if 'GAT MSE' in i[0]][1]
gat_kl=[te_train[i] for i in te_train if 'GAT KL Divergence' in i[0]][1]
train_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
train_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
train_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

gcn_error=[te_valid[i] for i in te_valid if 'GCN ATE % Error' in i[0]][2]
gcn_mse=[te_valid[i] for i in te_valid if 'GCN MSE' in i[0]][2]
gcn_kl=[te_valid[i] for i in te_valid if 'GCN KL Divergence' in i[0]][2]
gat_error=[te_valid[i] for i in te_valid if 'GAT ATE % Error' in i[0]][1]
gat_mse=[te_valid[i] for i in te_valid if 'GAT MSE' in i[0]][1]
gat_kl=[te_valid[i] for i in te_valid if 'GAT KL Divergence' in i[0]][1]
valid_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
valid_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
valid_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

print("Validation Summary of Embedding(10d) & Causal Weighting(6d)")
valid_summary

Validation Summary of Embedding(10d) & Causal Weighting(6d)


,Abs % Error of ATE,MSE,KL Divergence
S learner(GCN),0.042666,0.155125,0.433525
S learner(GAT),0.440617,0.090175,0.813079


In [ ]:
train_summary.to_csv("Train_embedding(10d)_ate(10d)_x9.csv")
valid_summary.to_csv("Valid_embedding(10d)_ate(10d)_x9.csv")